In [1]:
#!pip install uszipcode
from Util import GREEN_TAXI

from ipywidgets import HBox, Layout, VBox , Label
#import pprint
import ipywidgets as widgets
import plotly.express as px

### distance calculatore 
import haversine as hs


C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df=GREEN_TAXI.pd.read_csv("./data/updated_RAW_TAXI.csv")

In [3]:
df

,Unnamed: 0,lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Trip_distance,PICKUP_ZIP,DROP_ZIP,P_Day,P_WEEK_Day,P_Hour,P_MIN,flag
0,0,2016-02-01 00:00:01,2016-02-01 00:10:06,-73.9390000,40.8052000,-73.9725000,40.7859000,2.8600000,10035,10162.0000000,1,Monday,0,0,1
1,1,2016-02-01 00:01:33,2016-02-01 00:20:13,-73.8915000,40.7467000,-73.8909000,40.7439000,3.3500000,11372,11373.0000000,1,Monday,0,0,1
2,2,2016-02-01 00:03:46,2016-02-01 00:21:04,-73.9838000,40.6761000,-73.9570000,40.7183000,4.7000000,11217,11211.0000000,1,Monday,0,0,1
3,3,2016-02-01 00:00:05,2016-02-01 00:06:48,-73.8075000,40.7004000,-73.8317000,40.7060000,2.1100000,11435,11351.0000000,1,Monday,0,0,1
4,4,2016-02-01 00:06:20,2016-02-01 00:08:47,-73.9040000,40.7449000,-73.9000000,40.7336000,0.9800000,11377,11378.0000000,1,Monday,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506026,1510717,2016-02-29 23:00:49,2016-02-29 23:17:52,-73.8672000,40.7524000,-73.8779000,40.7565000,2.1000000,11369,11372.0000000,29,Monday,23,0,1
1506027,1510718,2016-02-29 23:00:46,2016-02-29 23:10:08,-73.9167000,40.7580000,-73.8820000,40.7559000,1.9000000,11103,11372.0000000,29,Monday,23,0,1
1506028,1510719,2016-02-29 23:00:17,2016-02-29 23:15:02,-73.9794000,40.6883000,-73.9236000,40.6989000,3.3000000,11217,11237.0000000,29,Monday,23,0,1
1506029,1510720,2016-02-29 23:00:02,2016-02-29 23:06:22,-73.9875000,40.7024000,-73.9950000,40.7199000,2.2000000,11201,10002.0000000,29,Monday,23,0,1


In [5]:
###GET Month-weekday_dict:
WEEK_DAY_ZIP=dict(zip(df["P_Day"].astype(int),df["P_WEEK_Day"])) 
#WEEK_DAY_ZIP

Merging Pickup - Drop Off location to get the density of taxi per each zip code area

In [6]:
df1=df[["lpep_pickup_datetime",'Pickup_longitude', 'Pickup_latitude','PICKUP_ZIP','P_Day',
       'P_WEEK_Day', 'P_Hour', 'P_MIN', 'flag']]

df1.columns=["datetime",'longitude', 'latitude','ZIP','P_Day',
       'P_WEEK_Day', 'P_Hour', 'P_MIN', 'flag']


df2=df[['Lpep_dropoff_datetime','Dropoff_longitude','Dropoff_latitude', 'DROP_ZIP','P_Day',
       'P_WEEK_Day', 'P_Hour', 'P_MIN', 'flag']]
df2.columns=["datetime",'longitude', 'latitude','ZIP','P_Day',
       'P_WEEK_Day', 'P_Hour', 'P_MIN', 'flag']


df=GREEN_TAXI.pd.concat([df1,df2])
df["ZIP"]=df["ZIP"].astype(int)
del(df1)
del(df2)

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3012062 entries, 0 to 1506030
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   datetime    object 
 1   longitude   float64
 2   latitude    float64
 3   ZIP         int32  
 4   P_Day       int64  
 5   P_WEEK_Day  object 
 6   P_Hour      int64  
 7   P_MIN       int64  
 8   flag        int64  
dtypes: float64(2), int32(1), int64(4), object(2)
memory usage: 218.3+ MB
None


,datetime,longitude,latitude,ZIP,P_Day,P_WEEK_Day,P_Hour,P_MIN,flag
0,2016-02-01 00:00:01,-73.9390000,40.8052000,10035,1,Monday,0,0,1
1,2016-02-01 00:01:33,-73.8915000,40.7467000,11372,1,Monday,0,0,1
2,2016-02-01 00:03:46,-73.9838000,40.6761000,11217,1,Monday,0,0,1
3,2016-02-01 00:00:05,-73.8075000,40.7004000,11435,1,Monday,0,0,1
4,2016-02-01 00:06:20,-73.9040000,40.7449000,11377,1,Monday,0,0,1


In [7]:
tem_df=df.groupby(['ZIP','P_Day', 'P_Hour'])["flag"].sum().reset_index()

In [8]:
print(tem_df.info())
tem_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107199 entries, 0 to 107198
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   ZIP     107199 non-null  int64
 1   P_Day   107199 non-null  int64
 2   P_Hour  107199 non-null  int64
 3   flag    107199 non-null  int64
dtypes: int64(4)
memory usage: 3.3 MB
None


,ZIP,P_Day,P_Hour,flag
0,7001,20,0,1
1,7002,4,19,1
2,7002,6,4,1
3,7002,13,0,1
4,7002,17,11,1
...,...,...,...,...
107194,11801,16,7,1
107195,11801,21,5,6
107196,11801,22,1,1
107197,11801,25,14,1


## Generate an empty data frame to calculate the probability of seeing Taxi in each zip code per weekday:    

In [9]:
ZIP=tem_df["ZIP"].unique().tolist()
DAY=tem_df["P_Day"].unique().tolist()
HR=tem_df["P_Hour"].unique().tolist()

def basae_file(ZIP,DAY,HR):
    data=[]
    for Zip in ZIP:
        for PDAY in DAY:
            for hr in HR:
                data.append([int(Zip),int(PDAY),int(hr),0,1])
                
    base_df=GREEN_TAXI.pd.DataFrame(columns=["ZIP","P_Day","P_Hour","TAXI","NO_TAXI"],data=data)
    return base_df



In [10]:
base_df= basae_file(ZIP,DAY,HR)
base_df

,ZIP,P_Day,P_Hour,TAXI,NO_TAXI
0,7001,20,0,0,1
1,7001,20,19,0,1
2,7001,20,4,0,1
3,7001,20,11,0,1
4,7001,20,17,0,1
...,...,...,...,...,...
273523,11801,25,8,0,1
273524,11801,25,9,0,1
273525,11801,25,12,0,1
273526,11801,25,7,0,1


## Merge the bases empty dataset to aggregate data

In [11]:
total = GREEN_TAXI.pd.merge(base_df, tem_df, on=["ZIP","P_Day","P_Hour"], how='left')
total

,ZIP,P_Day,P_Hour,TAXI,NO_TAXI,flag
0,7001,20,0,0,1,1.0000000
1,7001,20,19,0,1,NaN
2,7001,20,4,0,1,NaN
3,7001,20,11,0,1,NaN
4,7001,20,17,0,1,NaN
...,...,...,...,...,...,...
273523,11801,25,8,0,1,NaN
273524,11801,25,9,0,1,NaN
273525,11801,25,12,0,1,NaN
273526,11801,25,7,0,1,NaN


## Feature generation to calculate the probability of having a taxi or not

In [12]:
total["TOtal_Taxi"]=total["flag"]+total["TAXI"]
#total["P_Day"]=total["P_Day_x"].astype(int)
total["P_WEEK_Day"]=total.apply(lambda x: WEEK_DAY_ZIP[x.P_Day],axis=1)
total=total.fillna(0)
total



,ZIP,P_Day,P_Hour,TAXI,NO_TAXI,flag,TOtal_Taxi,P_WEEK_Day
0,7001,20,0,0,1,1.0000000,1.0000000,Saturday
1,7001,20,19,0,1,0.0000000,0.0000000,Saturday
2,7001,20,4,0,1,0.0000000,0.0000000,Saturday
3,7001,20,11,0,1,0.0000000,0.0000000,Saturday
4,7001,20,17,0,1,0.0000000,0.0000000,Saturday
...,...,...,...,...,...,...,...,...
273523,11801,25,8,0,1,0.0000000,0.0000000,Thursday
273524,11801,25,9,0,1,0.0000000,0.0000000,Thursday
273525,11801,25,12,0,1,0.0000000,0.0000000,Thursday
273526,11801,25,7,0,1,0.0000000,0.0000000,Thursday


In [13]:

total["TAXI"]=total.apply(lambda x: 1 if int(x.flag) > 0 else 0, axis=1)
total["NO_TAXI"]=total["NO_TAXI"]-total["TAXI"]
total["ID"]=1

## Aggregate data on a weekly day and calculate the probability

In [14]:
Model_data= (total.groupby(["ZIP","P_WEEK_Day","P_Hour"]).agg(
    {'TOtal_Taxi':'mean','TAXI':'sum', 'NO_TAXI': 'sum',"ID":"sum"}).reset_index().rename(
    columns={'TOtal_Taxi':'Average_Taxi'}))

###########################################################################
Model_data["Prob_at_least_one_taxi"]=Model_data["TAXI"]/Model_data["ID"]

### Probability of seeing Taxi = Our expectation * Probability of seeing one taxi 
Model_data["Prob"]=Model_data["Average_Taxi"]*Model_data["Prob_at_least_one_taxi"]


In [15]:
Model_data

,ZIP,P_WEEK_Day,P_Hour,Average_Taxi,TAXI,NO_TAXI,ID,Prob_at_least_one_taxi,Prob
0,7001,Friday,0,0.0000000,0,4,4,0.0000000,0.0000000
1,7001,Friday,1,0.0000000,0,4,4,0.0000000,0.0000000
2,7001,Friday,2,0.0000000,0,4,4,0.0000000,0.0000000
3,7001,Friday,3,0.0000000,0,4,4,0.0000000,0.0000000
4,7001,Friday,4,0.0000000,0,4,4,0.0000000,0.0000000
...,...,...,...,...,...,...,...,...,...
66019,11801,Wednesday,19,0.0000000,0,4,4,0.0000000,0.0000000
66020,11801,Wednesday,20,0.0000000,0,4,4,0.0000000,0.0000000
66021,11801,Wednesday,21,0.0000000,0,4,4,0.0000000,0.0000000
66022,11801,Wednesday,22,0.0000000,0,4,4,0.0000000,0.0000000


In [16]:
#### Poztalcod Finder based on User uolcation: 


In [17]:
###test
LAT="40.8052000"
LANG="-73.9390000"
GREEN_TAXI.ZIP_CODE(LAT,LANG)


'10035'

In [18]:
def Address_finder(LAT,LANG):
    USER_ZIP=GREEN_TAXI.ZIP_CODE(float(LAT),float(LANG))

    LAT=round(float(LAT),5)
    LANG=round(float(LANG),5)
    LIST_POSITION=[]
    for i in range(-3,3):
        LAT_I=0.001 * i
        for j in range(-3,3):
            LANG_J=0.001 * j
            
            tem=[LAT+LAT_I,LANG+LANG_J]            
            #print(tem[1])
            DIST=hs.haversine([LAT,LANG],tem,unit=hs.Unit.METERS)
            POST=GREEN_TAXI.ZIP_CODE(tem[0],tem[1])                        
            LIST_POSITION.append([POST,LAT+LAT_I,LANG+LANG_J,DIST])
    tem_pickup=GREEN_TAXI.pd.DataFrame(columns=["ZIP","LAT","LNG","Distance"],data=LIST_POSITION)
    tem_pickup=tem_pickup.sort_values(["Distance"])
    
    tem_pickup=tem_pickup.drop_duplicates(subset=["ZIP"],keep="first")
            
    return tem_pickup

LIST_POSITION=Address_finder(LAT,LANG)    
LIST_POSITION    
    

,ZIP,LAT,LNG,Distance
21,10035,40.8052000,-73.9390000,0.0000000
20,10037,40.8052000,-73.9400000,84.1675314
18,10026,40.8052000,-73.9420000,252.5025943


tem=LIST_POSITION.drop_duplicates(subset=["ZIP"],keep="first")
tem

In [43]:


class LOCATION_RECOMMENDER: 
    def __init__(self,df):
        self.df=df
        #self.DICT=DICT
        self.BTN_select = widgets.Button(description='Analysis')
        self.BTN_outt = widgets.Output()
        #self.tem_pickup= pd.DataFrame()#####
        self.tem_pickup= GREEN_TAXI.pd.DataFrame()#####
       
        def on_button_clicked(_):
            with self.BTN_outt:
                
                self.BTN_outt.clear_output()
                widgets.Output().clear_output()
                print("DAY : ",self.DAY, "Hour : "  , self.Hour,"LAT : ",self.USERLAT, "LNG :" ,self.USERLNG)
                #############################################################################################
                ### evaluating the inputs:
                
                try:
                    self.USERZIP=int(float(str(self.USERZIP)))
                except:
                    self.USERZIP=00000
                try:
                    self.USERLAT=abs(float(self.USERLAT))*1.000001
                except:
                    self.USERLAT=0.00000 
                try:
                    self.USERLNG=abs(float(self.USERLNG))*-1.000001
                except:
                    self.USERLAT=0.00000
                    
                    
                print(self.USERLNG, self.USERLAT ,self.USERZIP)
                ####MIN LNg-74.4786 MAX LNG-73.5017  MIN LAT 40.4256  MAX LAT 40.9996
                if -74.4786 <= self.USERLNG <= -73.5017 and 40.4256 <= self.USERLAT <=40.9996:
                    print("************looking for the closest dictance to the given GEO**************")
                    self.Address_finder()
                
                else:
                    try:
                        print("************looking for the closest dictance to the given ZIP CODE **************")
                        self.USERLAT,self.USERLNG=GREEN_TAXI.ZIP_to_points(self.USERZIP)
                        print(""" Extracting  info ..""",self.USERLAT,self.USERLNG)
                        self.Address_finder()
                    except:
                        print("""
                        please try again : 
                        LAT : 40.8062000
                        LNG : -73.9400000
                        ZIP : 10035
                        """ )
                if len(self.tem_pickup) >=1:
                    self.Recommender()

        ##############################################################################################33        
        def Chooser(DAY, Hour,USERZIP,USERLAT,USERLNG):

            HBOX_COMMENT=HBox([Label('GREEN TAXI Reporting System ....."')])

            self.BTN_select.on_click(on_button_clicked) #Function to run on click

            container2 = widgets.VBox([HBOX_COMMENT,self.BTN_select,self.BTN_outt])

            display(container2)
            
            self.DAY=DAY
            self.Hour=Hour
            self.USERZIP=USERZIP
            self.USERLAT=USERLAT
            self.USERLNG=USERLNG

        _ = widgets.interact(
                    Chooser, 
                    DAY=['Monday','Tuesday','Wednesday','Thursday', 'Friday','Saturday','Sunday'] ,
                    USERZIP="10035",
                    USERLAT="0.00000",
                    USERLNG="0.00000",
                    Hour=[x for x in range(24)],
                    #DICT=[DICT]
                    )                

        
    def Address_finder(self):
        #self.USER_ZIP=ZIP_CODE(float(self.USERLAT),float(self.USERLNG))
        self.USER_ZIP=GREEN_TAXI.ZIP_CODE(float(self.USERLAT),float(self.USERLNG))

        LAT=round(float(self.USERLAT),5)
        LANG=round(float(self.USERLNG),5)
        LIST_POSITION=[]
        for i in range(-3,3):
            LAT_I=0.001 * i
            for j in range(-3,3):
                LANG_J=0.001 * j
                tem=[LAT+LAT_I,LANG+LANG_J]            
                #print(tem[1])
                DIST=hs.haversine([LAT,LANG],tem,unit=hs.Unit.METERS)
                #POST=ZIP_CODE(tem[0],tem[1]) #### model
                try:
                    POST=GREEN_TAXI.ZIP_CODE(tem[0],tem[1])

                    LIST_POSITION.append([int(float(POST)),LAT+LAT_I,LANG+LANG_J,DIST])
                except:
                    pass
        if len(LIST_POSITION) >=1:
        #self.tem_pickup=pd.DataFrame(columns=["ZIP","LAT","LNG","Distance"],data=LIST_POSITION)
            self.tem_pickup=GREEN_TAXI.pd.DataFrame(columns=["ZIP","LAT","LNG","Distance"],data=LIST_POSITION)
            self.tem_pickup=self.tem_pickup.sort_values(["Distance"])
            self.tem_pickup=self.tem_pickup.drop_duplicates(subset=["ZIP"],keep="first")


            self.tem_pickup["P_WEEK_Day"]=self.DAY
            self.tem_pickup["P_Hour"]=int(self.Hour)
        #print(self.tem_pickup)
        else:
            self.tem_pickup=GREEN_TAXI.pd.DataFrame(columns=["ZIP","LAT","LNG","Distance"])
            
        

        #tem_pickup


        #return tem_pickup
    def Recommender(self):
        self.report=GREEN_TAXI.pd.merge(self.tem_pickup,self.df,how="left",on=["ZIP","P_WEEK_Day","P_Hour"])
        if len(self.report)>=1:
            #print(self.report)
            self.plot()
        
        
    def plot(self):
        print("Loading User Location ......")
        ZIP=self.report["ZIP"]
        Distance=self.report["Distance"]
        AVG=self.report["Average_Taxi"]
        PROB=self.report["Prob"]
        PR=self.report["Prob_at_least_one_taxi"]

        print("best location according to the on month observations: ")
        for i in range (len(test.report)) :
             print ("Location :",ZIP[i]," Dictance to the user in Meter: ", Distance[i] ," expected TAXI: ", AVG[i] ,"with Probaility of :",PR[i],"Probability * AVerage = Expectation: ",PROB[i]  )

        """
        ZIP        LAT         LNG   Distance   P_WEEK_Day  P_Hour  Average_Taxi  \
        10035 40.8000000 -73.9300000 0.0000000     Monday       0    54.6000000   

           TAXI  NO_TAXI  ID  Prob_at_least_one_taxi       Prob  
        0     5       -5   5        1.0000000          54.6000000  
        """

        fig = px.scatter_mapbox(self.report, lat="LAT", lon="LNG", hover_name="ZIP", hover_data=["Distance", "P_WEEK_Day",
                                                                                               "Average_Taxi","Prob","Prob_at_least_one_taxi",
                                                                                                "ID"],
                                color_discrete_sequence=["orange"], size="Average_Taxi",zoom=15, height=500)



        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.show()
        
        





In [44]:
test=LOCATION_RECOMMENDER(Model_data)

interactive(children=(Dropdown(description='DAY', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Frid…

In [45]:
test.report

,ZIP,LAT,LNG,Distance,P_WEEK_Day,P_Hour,Average_Taxi,TAXI,NO_TAXI,ID,Prob_at_least_one_taxi,Prob
0,10007,40.7140000,-74.0080000,0.0000000,Saturday,18,6.5000000,4,0,4,1.0000000,6.5000000
1,10278,40.7150000,-74.0060000,201.9368093,Saturday,18,9.7500000,4,0,4,1.0000000,9.7500000
2,10271,40.7110000,-74.0090000,344.0684116,Saturday,18,2.5000000,4,0,4,1.0000000,2.5000000
3,10038,40.7110000,-74.0060000,373.7578748,Saturday,18,6.2500000,4,0,4,1.0000000,6.2500000
